<a href="https://colab.research.google.com/github/Ayman850/COMP257/blob/main/COMP257_Ass3_AymanYasin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment3 - Question 1

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.cluster import KMeans, DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
from scipy.stats import mode

## 1. Retrieve and load the Olivetti faces dataset

In [ ]:
olivetti_faces = fetch_olivetti_faces()
X = olivetti_faces.data
y = olivetti_faces.target
images = olivetti_faces.images # 64x64 version of .data
print(X.shape)
print(y.shape)

In [ ]:
# From Reference [1]
def show_40_distinct_people(images, unique_ids):
    #Creating 4X10 subplots in  18x9 figure size
    fig, axarr=plt.subplots(nrows=4, ncols=10, figsize=(18, 9))
    #For easy iteration flattened 4X10 subplots matrix to 40 array
    axarr=axarr.flatten()

    #iterating over user ids
    for unique_id in unique_ids:
        image_index=unique_id*10
        axarr[unique_id].imshow(images[image_index], cmap='gray')
        axarr[unique_id].set_xticks([])
        axarr[unique_id].set_yticks([])
        axarr[unique_id].set_title("face id:{}".format(unique_id))
    plt.suptitle("There are 40 distinct people in the dataset")

show_40_distinct_people(images, np.unique(y))

## 2. Split the training set, a validation set, and a test set using stratified sampling to ensure that there are the same number of images per person in each set.

In [ ]:
'''
Split is:
80% train
10% validation
10% test
to ensure we have a lot of data to train,
and also still have some left for validation and testing
'''
# First split into train + validation (90%) and test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

# Now split the train + validation set (90%) into train (80%) and validation (10%)
# 10/90 = 0.1111
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

## 3. Using k-fold cross validation, train a classifier to predict which person is represented in each picture, and evaluate it on the validation set.

In [ ]:
# Use SVC with linear kernel
clf = SVC(kernel='linear', random_state=42)

# Instantiate StratifiedKFold
# use n_splits=4 so the 8 training samples per class can be evenly distributed
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Run k-fold CV
cv_scores = cross_val_score(clf, X_train, y_train, cv=kf, scoring='accuracy')

# Print scores
print(f"Cross-validation accuracy scores: {cv_scores}")
print(f"Mean cross-validation accuracy: {cv_scores.mean():.4f}")

# Train the classifier on the entire training set
clf.fit(X_train, y_train)

# Evaluate the classifier on the validation set
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation set accuracy: {val_accuracy:.4f}")

## 4. Using either Agglomerative Hierarchical Clustering (AHC) or Divisive Hierarchical Clustering (DHC) and using the centroid-based clustering rule, reduce the dimensionality of the set by using the following similarity measures:

In [ ]:
n_clusters = 40

In [ ]:
# From Reference [2]
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)
    title = kwargs.pop('title', "")
    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        # plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.title(title)
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

# Function to simplify code to use fancy_dendrogram()
def use_fancy_dendrogram(Z, title, filename):
  plt.figure(figsize=(25, 10))
  fancy_dendrogram(Z,
                 leaf_rotation=90,
                 leaf_font_size=12,
                 show_contracted=True,
                 title=title
                )
  plt.savefig(filename)
  plt.show()

In [ ]:
def get_centroids(X, labels):
  unique_labels = np.unique(labels)
  centroids_list = []
  for i in unique_labels:
    # Take points that belong to cluster i
    cluster_points = X[labels == i]

    # Calculate the mean points of the points in the cluster
    centroid = cluster_points.mean(axis=0)

    # Add to list
    centroids_list.append(centroid)
  centroids = np.array(centroids_list)
  return centroids

### a) Euclidean Distance

In [ ]:
# Use euclidean distance
dist_euclidean = pdist(X, metric="euclidean")

# Perform agglomerative clustering
Z_euclidean = linkage(dist_euclidean, method="centroid")

# Get labels (which cluster each point belongs to)
labels_euclidean = fcluster(Z_euclidean, t=n_clusters, criterion="maxclust")

In [ ]:
# Show how data points are merged
use_fancy_dendrogram(Z_euclidean, f"dendrogram_euclidean", f"dendrogram_euclidean.png")

In [ ]:
# Get reduced dataset
X_reduced_euclidean = get_centroids(X, labels_euclidean)
X_reduced_euclidean.shape

### b) Minkowski Distance

In [ ]:
# Use minkowski distance (if p=2, equivalent to Euclidean)
# p=1 is Manhattan
dist_minkowski = pdist(X, metric="minkowski", p=1)

# Perform agglomerative clustering
Z_minkowski = linkage(dist_minkowski, method="centroid")

# Get labels (which cluster each point belongs to)
labels_minkowski = fcluster(Z_minkowski, t=n_clusters, criterion="maxclust")

In [ ]:
# Show how data points are merged
use_fancy_dendrogram(Z_minkowski, f"dendrogram_minkowski_p1", f"dendrogram_minkowski_p1.png")

In [ ]:
# Get reduced dataset
X_reduced_minkowski = get_centroids(X, labels_minkowski)
X_reduced_minkowski.shape

### c) Cosine Similarity

Methods ‘centroid’, ‘median’, and ‘ward’ are correctly defined only if Euclidean pairwise metric is used. If y is passed as precomputed pairwise distances, then it is the user’s responsibility to assure that these distances are in fact Euclidean, otherwise the produced result will be incorrect.  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

In [ ]:
# Use cosine similarity
dist_cosine = pdist(X, metric="cosine")

# Perform agglomerative clustering
Z_cosine = linkage(dist_cosine, method="average")

# Get labels (which cluster each point belongs to)
labels_cosine = fcluster(Z_cosine, t=n_clusters, criterion="maxclust")

In [ ]:
# Show how data points are merged
use_fancy_dendrogram(Z_cosine, f"dendrogram_cosine", f"dendrogram_cosine.png")

In [ ]:
# Get reduced dataset
X_reduced_cosine = get_centroids(X, labels_cosine)
X_reduced_cosine.shape

## 5. Discuss any discrepancies observed between 4(a), 4(b), or 4(c). Use the silhouette score approach to choose the number of clusters for 4(a), 4(b), and 4(c).

In [ ]:
cluster_range = range(20, 150)
silhouette_scores = {
    "euclidean": [],
    "minkowski": [],
    "cosine": []
}

for c in cluster_range:
  print(f"c: {c}")
  labels_euclidean = fcluster(Z_euclidean, t=c, criterion="maxclust")
  labels_minkowski = fcluster(Z_minkowski, t=c, criterion="maxclust")
  labels_cosine = fcluster(Z_cosine, t=c, criterion="maxclust")

  sil_euclidean = silhouette_score(X, labels_euclidean, metric='euclidean')
  sil_minkowski = silhouette_score(X, labels_minkowski, metric='minkowski', p=1)
  sil_cosine = silhouette_score(X, labels_cosine, metric='cosine')

  silhouette_scores["euclidean"].append(sil_euclidean)
  silhouette_scores["minkowski"].append(sil_minkowski)
  silhouette_scores["cosine"].append(sil_cosine)

optimal_euclidean = cluster_range[np.argmax(silhouette_scores["euclidean"])]
optimal_minkowski = cluster_range[np.argmax(silhouette_scores["minkowski"])]
optimal_cosine = cluster_range[np.argmax(silhouette_scores["cosine"])]

print(f"Optimal number of clusters for euclidean: {optimal_euclidean}")
print(f"Optimal number of clusters for minkowski (p=1): {optimal_minkowski}")
print(f"Optimal number of clusters for cosine: {optimal_cosine}")

In [ ]:
# Plot silhouette scores to visualize the optimal number of clusters
plt.figure(figsize=(8, 5))
plt.plot(cluster_range, silhouette_scores["euclidean"], marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters (Euclidean)')
plt.grid(True)
plt.savefig("silhouette_euclidean.png")
plt.show()

In [ ]:
# Plot silhouette scores to visualize the optimal number of clusters
plt.figure(figsize=(8, 5))
plt.plot(cluster_range, silhouette_scores["minkowski"], marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters (Minkowski p=1)')
plt.grid(True)
plt.savefig("silhouette_minkowski_p1.png")
plt.show()

In [ ]:
# Plot silhouette scores to visualize the optimal number of clusters
plt.figure(figsize=(8, 5))
plt.plot(cluster_range, silhouette_scores["cosine"], marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters (Cosine Similarity)')
plt.grid(True)
plt.savefig("silhouette_cosine.png")
plt.show()

## 6. Use the set from (4(a), 4(b), or 4(c)) to train a classifier as in (3) using k-fold cross validation.

In [ ]:
print(f"Optimal number of clusters for euclidean: {optimal_euclidean}")
print(f"Optimal number of clusters for minkowski: {optimal_minkowski}")
print(f"Optimal number of clusters for cosine: {optimal_cosine}")

In [ ]:
# Use euclidean distance
dist_euclidean = pdist(X, metric="euclidean")

# Perform agglomerative clustering
Z_euclidean = linkage(dist_euclidean, method="centroid")

# Get labels (which cluster each point belongs to)
labels_euclidean = fcluster(Z_euclidean, t=optimal_euclidean, criterion="maxclust")

In [ ]:
# Use minkowski distance (if p=2, equivalent to Euclidean)
# p=1 is Manhattan
dist_minkowski = pdist(X, metric="minkowski", p=1)

# Perform agglomerative clustering
Z_minkowski = linkage(dist_minkowski, method="centroid")

# Get labels (which cluster each point belongs to)
labels_minkowski = fcluster(Z_minkowski, t=optimal_minkowski, criterion="maxclust")

In [ ]:
# Use cosine similarity
dist_cosine = pdist(X, metric="cosine")

# Perform agglomerative clustering
Z_cosine = linkage(dist_cosine, method="average")

# Get labels (which cluster each point belongs to)
labels_cosine = fcluster(Z_cosine, t=optimal_cosine, criterion="maxclust")

In [ ]:
def get_y_reduced(y, labels):
  y_reduced = []
  for i in np.unique(labels):
    cluster_indices = np.where(labels == i)

    # Get labels of points within this cluster
    cluster_labels = y[cluster_indices]

    # Get the label with most occurence for this cluster
    cluster_label = mode(cluster_labels).mode

    y_reduced.append(cluster_label)
  return np.array(y_reduced)

In [ ]:
# Get reduced Xs
X_reduced_euclidean = get_centroids(X, labels_euclidean)
X_reduced_minkowski = get_centroids(X, labels_minkowski)
X_reduced_cosine = get_centroids(X, labels_cosine)

# Get reduced ys
y_reduced_euclidean = get_y_reduced(y, labels_euclidean)
y_reduced_minkowski = get_y_reduced(y, labels_minkowski)
y_reduced_cosine = get_y_reduced(y, labels_cosine)

# Verify shapes
print(X_reduced_euclidean.shape)
print(y_reduced_euclidean.shape)
print(X_reduced_minkowski.shape)
print(y_reduced_minkowski.shape)
print(X_reduced_cosine.shape)
print(y_reduced_cosine.shape)

In [ ]:
# do train val test split like previously
# For Euclidean
X_train_val_euclidean, X_test_euclidean, y_train_val_euclidean, y_test_euclidean = train_test_split(
    X_reduced_euclidean, y_reduced_euclidean, test_size=0.1, random_state=42)

X_train_euclidean, X_val_euclidean, y_train_euclidean, y_val_euclidean = train_test_split(
    X_train_val_euclidean, y_train_val_euclidean, test_size=0.1111, random_state=42)

# For Minkowski
X_train_val_minkowski, X_test_minkowski, y_train_val_minkowski, y_test_minkowski = train_test_split(
    X_reduced_minkowski, y_reduced_minkowski, test_size=0.1, random_state=42)

X_train_minkowski, X_val_minkowski, y_train_minkowski, y_val_minkowski = train_test_split(
    X_train_val_minkowski, y_train_val_minkowski, test_size=0.1111, random_state=42)

# For Cosine Similarity
X_train_val_cosine, X_test_cosine, y_train_val_cosine, y_test_cosine = train_test_split(
    X_reduced_cosine, y_reduced_cosine, test_size=0.1, random_state=42)

X_train_cosine, X_val_cosine, y_train_cosine, y_val_cosine = train_test_split(
    X_train_val_cosine, y_train_val_cosine, test_size=0.1111, random_state=42)

In [ ]:
print("Euclidean Shapes:")
print("X_train_euclidean:", X_train_euclidean.shape)
print("y_train_euclidean:", y_train_euclidean.shape)
print("X_val_euclidean:", X_val_euclidean.shape)
print("y_val_euclidean:", y_val_euclidean.shape)
print("X_test_euclidean:", X_test_euclidean.shape)
print("y_test_euclidean:", y_test_euclidean.shape)
print()
print("Minkowski (p=1) Shapes:")
print("X_train_minkowski:", X_train_minkowski.shape)
print("y_train_minkowski:", y_train_minkowski.shape)
print("X_val_minkowski:", X_val_minkowski.shape)
print("y_val_minkowski:", y_val_minkowski.shape)
print("X_test_minkowski:", X_test_minkowski.shape)
print("y_test_minkowski:", y_test_minkowski.shape)
print()
print("Cosine Shapes:")
print("X_train_cosine:", X_train_cosine.shape)
print("y_train_cosine:", y_train_cosine.shape)
print("X_val_cosine:", X_val_cosine.shape)
print("y_val_cosine:", y_val_cosine.shape)
print("X_test_cosine:", X_test_cosine.shape)
print("y_test_cosine:", y_test_cosine.shape)

In [ ]:
# Use SVC with linear kernel
clf = SVC(kernel='linear', random_state=42)

# Instantiate StratifiedKFold
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Use SVC with linear kernel for Euclidean reduced dataset
cv_scores_euclidean = cross_val_score(clf, X_train_euclidean, y_train_euclidean, cv=kf, scoring='accuracy')
print(f"Cross-Validation Accuracy (Euclidean): {cv_scores_euclidean}")
print(f"Mean Accuracy (Euclidean): {cv_scores_euclidean.mean():.4f}")

# Use SVC with linear kernel for Minkowski reduced dataset
cv_scores_minkowski = cross_val_score(clf, X_train_minkowski, y_train_minkowski, cv=kf, scoring='accuracy')
print(f"Cross-Validation Accuracy (Minkowski p=1): {cv_scores_minkowski}")
print(f"Mean Accuracy (Minkowski p=1): {cv_scores_minkowski.mean():.4f}")

# Use SVC with linear kernel for Cosine reduced dataset
cv_scores_cosine = cross_val_score(clf, X_train_cosine, y_train_cosine, cv=kf, scoring='accuracy')
print(f"Cross-Validation Accuracy (Cosine): {cv_scores_cosine}")
print(f"Mean Accuracy (Cosine): {cv_scores_cosine.mean():.4f}")

In [ ]:
# Train the classifier on the entire Euclidean training set
clf.fit(X_train_euclidean, y_train_euclidean)

# Evaluate the classifier on the Euclidean validation set
y_val_pred_euclidean = clf.predict(X_val_euclidean)
val_accuracy_euclidean = accuracy_score(y_val_euclidean, y_val_pred_euclidean)

print(f"Validation set accuracy (Euclidean): {val_accuracy_euclidean:.4f}")

# Train the classifier on the entire Minkowski training set
clf.fit(X_train_minkowski, y_train_minkowski)

# Evaluate the classifier on the Minkowski validation set
y_val_pred_minkowski = clf.predict(X_val_minkowski)
val_accuracy_minkowski = accuracy_score(y_val_minkowski, y_val_pred_minkowski)

print(f"Validation set accuracy (Minkowski p=1): {val_accuracy_minkowski:.4f}")

# Train the classifier on the entire Cosine training set
clf.fit(X_train_cosine, y_train_cosine)

# Evaluate the classifier on the Cosine validation set
y_val_pred_cosine = clf.predict(X_val_cosine)
val_accuracy_cosine = accuracy_score(y_val_cosine, y_val_pred_cosine)

print(f"Validation set accuracy (Cosine): {val_accuracy_cosine:.4f}")

## References  
[1] https://www.kaggle.com/code/serkanpeldek/face-recognition-on-olivetti-dataset  
[2] https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/